<a href="https://colab.research.google.com/github/Chevron9/iannwtf/blob/Roho7/7/Exercise07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [ ]:
#loads the dataset and splits into test and training data
train_dataset = tfds.load('fashion_mnist', split="train[:100%]", as_supervised=True)
test_dataset = tfds.load('fashion_mnist', split="test[:100%]", as_supervised=True)

#normalize the images and make the labels a one hot vector
train_dataset = train_dataset.map(lambda x,y:((tf.image.per_image_standardization(
    tf.cast(x, tf.float32))), (tf.reshape(tf.one_hot(y, 10), (-1,)))))
test_dataset = test_dataset.map(lambda x,y:((tf.image.per_image_standardization(
     tf.cast(x, tf.float32))), (tf.reshape(tf.one_hot(y, 10), (-1,)))))


#creats numpy array to print 
train_numpy= tfds.as_numpy(train_dataset)
test_numpy = tfds.as_numpy(test_dataset)
print(len(train_dataset))
print(len(test_dataset))

#looks if the format is still correct
for i in train_numpy:
  print((i[0]).shape)
  break
for i in test_numpy:
  print((i[0]).shape)
  break

60000
10000
(28, 28, 1)
(28, 28, 1)


In [ ]:
#shuffel, prefetch and batch the datasets
train_dataset = train_dataset.batch(64).shuffle(buffer_size=64).prefetch(128)
test_dataset = test_dataset.batch(64).shuffle(buffer_size=64).prefetch(128)

In [ ]:
#Encoder takes the input and makes an embedding
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
class Encoder(Layer):
  def __init__(self, input_shape):
    super(Encoder, self).__init__()
    self.relu = tf.keras.activations.relu

    self.conv_layer_1= tf.keras.layers.Conv2D(filters= 64, kernel_size= 2,
                                                    strides=(2, 2), padding='valid',
                                                    input_shape = input_shape
                                                     )
        
    self.batch_normalization_layer_1 = tf.keras.layers.BatchNormalization()



    self.conv_layer_2= tf.keras.layers.Conv2D(filters = 32, kernel_size= 3,
                                                    strides=(2, 2), padding='valid'
                                                    )
    
    self.batch_normalization_layer_2 = tf.keras.layers.BatchNormalization()


  
    self.global_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
    self.output_layer = tf.keras.layers.Dense(units=10, activation=tf.keras.activations.sigmoid)

  def call(self, x, training = True):

    x = self.conv_layer_1(x)
    x = self.batch_normalization_layer_1(x, training)
    x = self.relu(x)

    x = self.conv_layer_2(x)
    x = self.batch_normalization_layer_2(x, training)
    x = self.relu(x)

    x = self.global_pooling_layer(x)
    x = self.output_layer(x)
    return x


In [ ]:
# takes embedding and generates original input
class Decoder(Layer):
 def __init__(self, input_shape):

  
  super(Decoder, self).__init__()
  self.dense = tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax)
  self.relu = tf.keras.activations.relu

  self.conv_layer_1= tf.keras.layers.Conv2DTranspose(filters= 32, kernel_size= 2,
                                                  strides=(2, 2), padding='valid',
                                                  input_shape = input_shape
                                                    )
      
  self.batch_normalization_layer_1 = tf.keras.layers.BatchNormalization()

  self.conv_layer_2= tf.keras.layers.Conv2DTranspose(filters = 64, kernel_size= 2,
                                                  strides=(2, 2), padding='valid'
                                                  )
  
  self.batch_normalization_layer_2 = tf.keras.layers.BatchNormalization()


  



  self.output_layer = tf.keras.layers.Conv2D(filters= 1, kernel_size= 3,
                                                  strides=(1, 1), padding='same', activation = tf.keras.activations.sigmoid
                                                  )

  def call(self, x, training = True):

    x = self.conv_layer_1(x)
    x = self.batch_normalization_layer_1(x, training)
    x = self.relu(x)

    x = self.conv_layer_2(x)
    x = self.batch_normalization_layer_2(x, training)
    x = self.relu(x)

    x = self.output_layer(x)

    return x

In [ ]:
#combines Encoder and Decoder to an Autoencoder
class Autoencoder(Model):
  def __init__(self):
    super(Autoencoder, self).__init__()

    self.encoder = Encoder((64,28,28,1))
    self.decoder = Decoder((64,28,28,10))
  
  def call(self, x, training = True):
    x = self.encoder(x, training)
    x = self.decoder(x, training)
    return x


In [ ]:
def train_step(model, input, target, loss_function, optimizer):
  # loss_object and optimizer_object are instances of respective tensorflow classes
  with tf.GradientTape() as tape:
    prediction = model(input)
    loss = loss_function(target, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

def test(model, test_data, loss_function):
  # test over complete test data

  test_accuracy_aggregator = []
  test_loss_aggregator = []

  for (input, target) in test_data:
    prediction = model(input, training = False)
    #loss between input and output of the autoencoder
    sample_test_loss = loss_function(input, prediction)
    sample_test_accuracy =  np.argmax(input, axis=1) == np.argmax(prediction, axis=1)
    sample_test_accuracy = np.mean(sample_test_accuracy)

    test_loss_aggregator.append(sample_test_loss.numpy())
    test_accuracy_aggregator.append(np.mean(sample_test_accuracy))

  test_loss = np.mean(test_loss_aggregator)
  test_accuracy = np.mean(test_accuracy_aggregator)

  return test_loss, test_accuracy
  

In [ ]:
tf.keras.backend.clear_session()

### Hyperparameters
num_epochs = 30
learning_rate = 0.00006
running_average_factor = 0.95

# Initialize the model.
model = Autoencoder()
# Initialize the loss: categorical cross entropy. Check out 'tf.keras.losses'.
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer: Adam with default parameters. Check out 'tf.keras.optimizers'
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Initialize lists for later visualization.
train_losses = []

test_losses = []
test_accuracies = []

#testing once before we begin
test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

#check how model performs on train data once before we begin
train_loss, _ = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)

#We train for num_epochs epochs.
for epoch in range(num_epochs):
    print('Epoch: __ ' + str(epoch))

    #training (and checking in with training)
    running_average = 0
    for (input,target) in train_dataset:
        # loss between output and input
        train_loss = train_step(model, input, input, cross_entropy_loss, optimizer)
        running_average = running_average_factor * running_average  + (1 - running_average_factor) * train_loss
    train_losses.append(running_average)

    #testing
    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    

TypeError: ignored